<img src="https://ai4future.keep.edu.hk/wp-content/uploads/2020/02/CUHK_AI_Logo_Update_Web.png" width="200" align="right"> <br><br><br><br>
<h1><span style="color:#338DFF"> AI With Python Workshop </span></h1>

### Welcome to the AI with Python Workshop by CUHK-Jockey Club AI for the Future Project

This notebook complements the powerpoint slides during the workshop and will be used to do the coding exercises 

-----------




# AI Project using Python

In this project we will create a virtual assistant that understands natural language. We will be able to interact with it in English to perform the following tasks:

<img src="https://cdn.dribbble.com/users/324481/screenshots/7149521/media/4fb25c016884dbbd1352e75916fa5f81.png?compress=1&resize=400x300" width="200" align="right">

* **Chit-chat**
    * Tell jokes
* **Weather**
    * Get current weather for any city
* **Movies**
    * Get rating for a movie
    * Find the director(s) of a movie
    * Find the actor(s) in a movie

## Learning outcomes

In this tutorial you will learn:  
* The basic terminologies required in virtual assistant systems
    * Intents
    * Slots
    * Entities
    * Utterances
* How to use SNIPS-NLU to understand natural language and detect intents, slots, and entities from utterances.
* How to use the detected intents, slots and entities to get information from APIs
----
----
----

# Let's Begin


We will use the following libraries in this tutorial 

* **snips-nlu**
    * This library deals with the Natural Language Understanding to detect intents, slots, and entities.
* **pyjokes**
    * This library provides jokes based on Python.
* **pyowm**
    * This library provides the weather API to get weather information.
* **imdbpy**
    * This library provides the IMDB movie APU to get movie information.

---

## Prepare the training dataset

Prepare the training dataset to train your Natural Language Understanding (NLU) Engine.

In our training dataset, we will have the following **intents**:

1. **tell_joke** : To detect that the user is asking the virtual assistant for a joke. There are no slots required for this intent.
    * **Example utterances:** "Hi, tell me a joke.", "I'm bored. Entertain me with a funny joke."  
    
    
2. **get_weather** : To detect that the user is asking for current weather of a city. For this intent we need to fill a slot for ``city``.
    * **Example utterances:** "How is the weather in New York?", "I wonder how the weather conditions are like in Hong Kong right now?"  
    
    
    
3. **get_rating** : To detect that the user is asking the rating for a movie. For this intent we need to fill a slot for ``movie_name``.
    * **Example utterances:** "How good is the movie Batman?", "I want to know the movie ratings for Fast and Furious"  
    
    
    
4. **get_director** : To detect that the user is asking for who is the director of a movie. For this intent we need to fill a slot for ``movie_name``.
    * **Example utterances:** "Who directed Tenet?", "I want to know the director of the movie Ip Man"  



5. **get_cast** : To detect that the user is asking for who acted in a movie. For this intent we need to fill a slot for ``movie_name``.
    * **Example utterances:** "Who acted in the movie Joker?", "What is the cast for the movie The Boat People?"
    

We will have the following **entities**:

1. **city**
    * **Examples**: Hong Kong, New York, Dublin, London  
    
    
2. **movie_name**
    * **Examples**: Star Wars, Ip Man, The Dark Knight, La la land


**We have created a starter dataset for you with 1 example intent and 1 example entity in the file ``dataset.yaml``**

## Install and Import libraries

Run the cell below to install and import the required libraries and functions.

**Note:** _We have pre-written some code to simplify the weather and movie rating APIs in the file ``utils.py``. You can view the file later to understand the inner working in more detail._

In [1]:
# Run this cell
!pip install pyjokes
!pip install snips-nlu
!pip install pyowm
!pip install imdbpy
!python -m snips_nlu download en
!git clone https://github.com/muditchaudhary/workshop_utils 
from workshop_utils.utils import *
import pyjokes
import json
from snips_nlu import SnipsNLUEngine
from snips_nlu.default_configs import CONFIG_EN

     |████████████████████████████████| 1.3 MB 5.3 MB/s 
  Created wheel for snips-nlu-en: filename=snips_nlu_en-0.2.3-py3-none-any.whl size=1297478 sha256=c6379d363907b73969784e935fb83c791655e7156cfd70976ce2c2acd3aa7e2d
  Stored in directory: /tmp/pip-ephem-wheel-cache-w3mq727z/wheels/77/e5/27/a2c7ae7b04c836360914a1ac909339da898cb66444e709f650
Successfully built snips-nlu-en
You should consider upgrading via the '/opt/python/envs/default/bin/python -m pip install --upgrade pip' command.

Linking successful
/opt/python/envs/default/lib/python3.8/site-packages/snips_nlu_en/snips_nlu_en-0.2.3 --> /opt/python/envs/default/lib/python3.8/site-packages/snips_nlu/data/en

Cloning into 'workshop_utils'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 17 (delta 5), reused 15 (delta 3), pack-reused 0
Unpacking objects: 100% (17/17), done.
You should consider upgrading via the '/opt/python/envs/d

## Convert the dataset to json format

Run the next cell to convert the dataset to json format to train the NLU Engine

In [3]:
# Run this cell

!snips-nlu generate-dataset en workshop_utils/dataset.yaml > dataset.json

# Open the dataset

To open the dataset, we will follow the following steps:
1. Use ``open`` function to load the file into Python in a variable called ``dataset_file``.
2. use ``load`` function from ``json`` as ``json.load(dataset_file)`` into a variable called ``training_dataset``.

In [4]:
# Write the code below
dataset_file = open("dataset.json", "r")
training_dataset = json.load(dataset_file)

## Initialize the Snips-NLU Engine with English Configuration

We will start our Snips-NLU engine using the ``SnipsNLUEngine()``. We will pass a parameter in it as ``config=CONFIG_EN``, which will load the English language configuration in our NLU engine. 

We will store the Snips-NLU engine in a variable called ``NLUengine``

In [5]:
# Write the code below

NLUengine = SnipsNLUEngine(config=CONFIG_EN)

## Train the NLU Engine

We will now train the NLU engine using our training dataset. We will use ``fit()`` function to train the model

To train the model we have to run:
```
NLUengine.fit(training_dataset)
```

In [6]:
# Write the code below

NLUengine.fit(training_dataset)

## Let's try to predict again

Let's try to use our model on the sentence ``"How's the weather in Hong Kong"``

Use the function ``prediction = NLUengine.parse(your utterance)``

In [8]:
# Write the code below

prediction = NLUengine.parse("How's the weather in Hong Kong?")

## Print the prediction

To print the prediction in a more readable format we will use ``json.dumps()`` function as:

``print(json.dumps(prediction, indent=2))``

In [9]:
# Write the code below

print(json.dumps(prediction, indent=2))

{
  "input": "How's the weather in Hong Kong?",
  "intent": {
    "intentName": "get_weather",
    "probability": 0.7324433420149711
  },
  "slots": [
    {
      "range": {
        "start": 21,
        "end": 30
      },
      "rawValue": "Hong Kong",
      "value": {
        "kind": "Custom",
        "value": "Hong Kong"
      },
      "entity": "city",
      "slotName": "city"
    }
  ]
}


## Get the intent

To get the intent we access the intent name element from the resulted ``prediction`` dictionary.

We have made a function for you to get the intent easily. You can use ``get_intent(prediction)`` to get the intent.

In [10]:
# Write the code below

print(get_intent(prediction))

get_weather


## Get the slot's entity type
There can be multiple slots mentioned in an utternace. But in our tutorial we only have 1 slot per utterance. 

You can use our function ``get_entity_type(prediction)`` to get the slot's entity type.

In [11]:
# Write the code below

print(get_entity_type(prediction))

city


## Get the entity's value

Similar to slots, there can be multiple entity values. But our tutorial will only have 1 value per slot. 

You can use our function ``get_entity_value(prediction)`` to get the slot's value.

In [12]:
# Write the code below

print(get_entity_value(prediction))

Hong Kong


## Integrate NLU Engine with API

We have provided you with the following pre-defined functions:
1. ```get_current_weather(city)```: Given a city, it will print its current temperature and weather condition
2. ```get_movie_rating(movie_name)```: Given a movie name, it will print its IMDB rating
3. ```get_directors(movie_name)```: Given a movie name, it will print the name(s) of its director(s)
4. ```get_cast(movie_name)```: Given a movie name, it will print the cast of the movie
5. ```pyjokes.get_joke()```: This function from pyjokes library returns a joke (a nerdy programming based joke)

We will now use these functions to integrate our NLU with the APIs to get a working virtual assistant

## Create a function

First we create a function called ```assistant``` that given an utterance, gives an appropriate response based on user's intent.

The function will have ``utterance`` as one of the parameter.

The function should work in the following manner:

1. Get the **intent** and **slot type** of the utterance using the NLU Engine
2. If the intent is ```tell_joke```, print the output of ```pyjokes.get_joke()``` function.
3. Else if the intent is ```get_weather```, get the value of slot ```city``` and use the ``get_current_weather(city)`` function.
4. Else if the intent is ```get_rating```, get the value of slot ```movie_name``` and use the ``get_movie_rating(movie_name)`` function.
5. Else if the intent is ```get_director```, get the value of slot ```movie_name``` and use the ``get_movie_directors(movie_name)`` function.
5. Else if the intent is ```get_cast```, get the value of slot ```movie_name``` and use the ``get_movie_cast(movie_name)`` function.
6. Else ``print("Unknown intent")``.

**Note: Inside the if-statements for ``get_weather``, ``get_rating``, ``get_director``, and ``get_cast``, you need to add another if-statement to check if the slot-type is correct. If it is not correct, you need to print "Sorry, please try again."**

In [13]:
# Write the code below

def assistant(utterance):
    prediction = NLUengine.parse(utterance)
    intent = get_intent(prediction)
    slot_type = get_entity_type(prediction)



    if (intent == "tell_joke"):
        print(pyjokes.get_joke())

    elif (intent == "get_weather"):

        if (slot_type == "city"): 
            city_name = get_entity_value(prediction)
            get_current_weather(city_name)
        else:
            print("Sorry, can you try again?")

    elif (intent == "get_rating"):

        if (slot_type == "movie_name"): 
            movie_name = get_entity_value(prediction)
            get_movie_rating(movie_name)
        else:
            print("Sorry, can you try again?")

    elif (intent == "get_director"):

        if (slot_type == "movie_name"): 
            movie_name = get_entity_value(prediction)
            get_movie_directors(movie_name)
        else:
            print("Sorry, can you try again?")

    elif (intent == "get_cast"):
        
        if (slot_type == "movie_name"): 
            movie_name = get_entity_value(prediction)
            get_movie_cast(movie_name)
        else:
            print("Sorry, can you try again?")

    else:
        print("Unknown intent")
        

## Create a conversation loop


We will create a loop that keeps on going until the user enters "Bye"

We will use a new kind of loop called ``while-loop``

The loop has to accomplish the following things:
1. Keep asking for user input until the user enters "Bye"
2. Call the ``assistant`` function on the user's input

To break the loop, we will use a new keyword called ``break``.  

To get input from the user we will use a Python function called ``input``.  

**We have partially written the code below to help you. Please fill the remaining code**

In [0]:

# Complete the code belo
print("Welcome to the virtual assistant. How can I help you?")
while True:
    print("------------------------------------")
    user_input=str(input("Enter your input: "))
    
    # This if statement should break the loop if the user_input is "Bye"
    if (user_input == "Bye"):
        print("Have a good day!")
        break
    
    else:
        
        print("Assistant: ")
        assistant(user_input)
        """Enter the code here to call assistant function using user_input here"""

Welcome to the virtual assistant. How can I help you?
------------------------------------
Enter your input:  Tell me a joke
Assistant: 
Child: Dad, why does the sun rise in the east and set in the west? Dad: Son, it's working, don't touch it.
------------------------------------
Enter your input:  How's the weather in Hong Kong?
Assistant: 
The current temperature is 32.59 degrees Celsius.
The weather condition is overcast clouds.
------------------------------------
Enter your input:  Who directed the movie Deadpool?
Assistant: 
The director(s) of the movie is/are:
Noriko Takao
------------------------------------
Enter your input:  Who directed Deadpool?\
Assistant: 
The director(s) of Deadpool? is/are:
David Leitch
------------------------------------
Enter your input:  Who acted in Deadpool?
Assistant: 
Sorry, can you try again?
------------------------------------
Enter your input:  Who acted in the movie Deadpool?
Assistant: 
The actors(s) in Deadpool? is/are:
Ryan Reynolds play

# End of Part 2

## Thank you for attending the workshop